In [187]:
import urllib.request  # импортируем модуль
import re
import urllib.parse
import os
import time
import requests
from requests.exceptions import HTTPError
from pymystem3 import Mystem

In [188]:
def take_Meta(html, url):  # извлекает мета-информацию
    meta = {}
    dating = ''
    pub = ''
    # ищет нужные данные для метаинформаци
    reg_posttitle = re.compile('<meta name="title" content="(.*?)"',
                                flags= re.DOTALL)
    date = re.compile('<span class="itemDateCreated">.*, (\d* [А-Яа-я]* 20.*) \d.*',
                                flags= re.DOTALL)
    authorship = re.compile('<meta name="author" content="(.*?)"',
                                flags= re.DOTALL)
    item = re.compile('Опубликовано в.*?<a.*?">(.*?)</a>',
                                flags= re.DOTALL)
    author = authorship.search(html)
    titles = reg_posttitle.search(html)
    title = titles[1]
    fin_date = date.search(html)
    category = item.search(html)
    clean_title = re.sub("&[A-Za-z]*;", "", title)
    cl_title = re.sub("[A-Za-z]*;", "", clean_title)
    # переводим дату создания в адекватный вид
    time_pub = fin_date[1].split(' ')
    if time_pub[1] != '':
        if time_pub[1] == "Январь":
            pub = "01"
        if time_pub[1] == "Февраль":
            pub = "02"
        if time_pub[1] == "Март":
            pub = "03"
        if time_pub[1] == "Апрель":
            pub = "04"
        if time_pub[1] == "Май":
            pub = "05"
        if time_pub[1] == "Июнь":
            pub = "06"
        if time_pub[1] == "Июль":
            pub = "07"
        if time_pub[1] == "Август":
            pub = "08"
        if time_pub[1] == "Сентябрь":
            pub = "09"
        if time_pub[1] == "Октябрь":
            pub = "10"
        if time_pub[1] == "Ноябрь":
            pub = "11"
        if time_pub[1] == "Декабрь":
            pub = "11"
    dating = str(time_pub[0]) + "." + str(pub) + '.' + str(time_pub[2])
    # создает словарь для бущуего удобного вытаскивания информаци
    if category == None:
        cat = "None"
        meta[cl_title] = [author[1], dating, cat, url, time_pub[2]]
    else:
        meta[cl_title] = [author[1], dating, category[1], url, time_pub[2]]
    return meta

In [189]:
def article_Text(html):  # функция чистит текст статьи
    # чистит текст
    finding_text = re.compile(
        '<div class="itemBody">.*<!-- Item introtext -->(.*)<!-- Item Rating -->', 
                            flags=re.DOTALL)
    text = finding_text.search(html)
    if text == None:
        final_t = ""
    else:
        final_t = str(text[1])
    regTag = re.compile('<.*?>', re.DOTALL)  # рег. выражение находит все тэги
    regScript = re.compile('<script>.*?</script>', re.DOTALL)  # все скрипты
    regComment = re.compile('<!--.*?-->', re.DOTALL)  # все комментарии
    specsymbols_1 = re.compile('&[A-Za-z]*;', re.DOTALL)  # специальные символы
    # а дальше заменяем ненужные куски на пустую строку
    clean_t = regScript.sub("  ", final_t)
    clean_t = regComment.sub("  ", clean_t)
    clean_t = regTag.sub("  ", clean_t)
    clean_text = specsymbols_1.sub("  ", clean_t)
    return clean_text

In [190]:
def textnTags(clean_text, meta, number):  # создает файл с текстом
    text = clean_text
    tags = ''
    # вытаскивает данные из словаря
    for key, value in meta.items():
        ti = key
        au = value[0]
        da = value[1]
        topic = value[2]
        url = value[3]
        year = value[4]
    dates = da.split('.')
    mounth = dates[1]
    # создает название файла
    filename = "article_" + str(number) + '.txt'
    # делает теги для записи в начале файла
    tags = "@au " + au + "\n@ti " + ti + "\n@da " + \
            da + "\n@topic " + topic + "\n@url " + url
    # создает название папки
    dirname = "plain"
    # прописвает пути
    path_dirname = "%s" % dirname
    if not os.path.exists(path_dirname):
        os.makedirs(path_dirname)
    dirname_year = "%s" % year
    path_year = os.path.join(path_dirname, dirname_year)
    if not os.path.exists(path_year):
        os.makedirs(path_year)
    dirname_mounth = "%s" % mounth
    path_mounth = os.path.join(path_year, dirname_mounth)
    if not os.path.exists(path_mounth):
        os.makedirs(path_mounth)
    # прописывает путь к файлу
    filepath = os.path.join(path_mounth, filename).encode('ascii')
    # создает файл
    with open(filepath, 'w', encoding="utf-8") as f:
        f.write(tags)
        clean = re.sub("  ", " ", text)
        f.write(clean)
        
        
    m = Mystem()
    lemmas = m.lemmatize(clean)
    new_clean = ''.join(lemmas)
    filename1 = "article_" + str(number) + '.txt'
    dirname1 = "mystem_lem"
    # прописвает пути
    path_dirname1 = "%s" % dirname1
    if not os.path.exists(path_dirname1):
        os.makedirs(path_dirname1)
    dirname_year1 = "%s" % year
    path_year1 = os.path.join(path_dirname1, dirname_year1)
    if not os.path.exists(path_year1):
        os.makedirs(path_year1)
    dirname_mounth1 = "%s" % mounth
    path_mounth1 = os.path.join(path_year1, dirname_mounth1)
    if not os.path.exists(path_mounth1):
        os.makedirs(path_mounth1)
    # прописывает путь к файлу
    filepath1 = os.path.join(path_mounth1, filename1).encode('ascii')
    with open(filepath1, 'w', encoding="utf-8") as f:
        f.write(new_clean)
        
    morf_text = m.analyze(clean)
    filename2 = "article_" + str(number) + '.txt'
    dirname2 = "mystem_plain"
    # прописвает пути
    path_dirname2 = "%s" % dirname2
    if not os.path.exists(path_dirname2):
        os.makedirs(path_dirname2)
    dirname_year2 = "%s" % year
    path_year2 = os.path.join(path_dirname2, dirname_year2)
    if not os.path.exists(path_year2):
        os.makedirs(path_year2)
    dirname_mounth2 = "%s" % mounth
    path_mounth2 = os.path.join(path_year2, dirname_mounth2)
    if not os.path.exists(path_mounth2):
        os.makedirs(path_mounth2)
    # прописывает путь к файлу
    filepath2 = os.path.join(path_mounth2, filename2).encode('ascii')
    with open(filepath2, 'w', encoding="utf-8") as f:
        f.write(str(morf_text))
    return filepath

In [191]:
def creating_Csv(filepath, meta):  # дополняет цсв-файл
    row = ''
    path = str(filepath)
    # достает информацию из словаря для записи в цсв
    for key, value in meta.items():
        author = value[0]
        header = key
        created = value[1]
        topic = value[2]
        source = value[3]
        year = value[4]
    # строка для цсв
    row = '%s\t%s\t%s\t%s\tпублицистика\t%s\tнейтральный\tн-возраст\tн-уровень\tрайонная\t%s\tВолжская правда\t%s\tгазета\tРоссия\tВолжский\tru\n'
    # запись в цсв-файл
    with open('meta.csv', 'a+', encoding='utf-8') as file:
        file.write(row % (path, author, header, created, topic, source, year))
    return

In [192]:
def main():  # запускающая функция
    row = '%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n'
    # создает цсв-файл
    with open('meta_1.csv', 'w', encoding='utf-8') as file:
        file.write(row % ("path", "author", "header",
                          "created", "sphere", "topic", "style",
                          "audience_age", "audience_level", "audience_size",
                          "source", "publication", "publ_year", "medium",
                          "country", "region", "language"))
    # подает ссылку к статье
    commonUrl = 'http://gazeta-vp.ru/news/./item/'
    for i in range(16700, 16730):  # ищет статьи в диапозоне
        pageUrl = commonUrl + str(i)
        try:
            r = requests.get(pageUrl)
            r.raise_for_status()
        except HTTPError:  # если выдается ошибка 404 - не читается старница
            print (r.url, 'Could not download page')
        else:  # основаня работа с данными
            print (r.url, 'downloaded successfully')
            url = r.url
            number = i
            # притворяемся браузером
            user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
            req = urllib.request.Request(url, headers={'User-Agent':user_agent})
            with urllib.request.urlopen(req) as response:
                html = response.read().decode('utf-8')
            take = take_Meta(html, url)
            text = article_Text(html)
            text_with_tg = textnTags(text, take, number)
            csv = creating_Csv(text_with_tg, take)
main()

http://gazeta-vp.ru/news/./item/16700 downloaded successfully
http://gazeta-vp.ru/news/./item/16701 downloaded successfully
http://gazeta-vp.ru/news/./item/16702 downloaded successfully
http://gazeta-vp.ru/news/./item/16703 downloaded successfully
http://gazeta-vp.ru/news/./item/16704 downloaded successfully
http://gazeta-vp.ru/news/./item/16705 downloaded successfully
http://gazeta-vp.ru/news/./item/16706 downloaded successfully
http://gazeta-vp.ru/news/./item/16707 downloaded successfully
http://gazeta-vp.ru/news/./item/16708 downloaded successfully
http://gazeta-vp.ru/news/./item/16709 Could not download page
http://gazeta-vp.ru/news/./item/16710 downloaded successfully
http://gazeta-vp.ru/news/./item/16711 downloaded successfully
http://gazeta-vp.ru/news/./item/16712 downloaded successfully
http://gazeta-vp.ru/news/./item/16713 downloaded successfully
http://gazeta-vp.ru/news/./item/16714 downloaded successfully
http://gazeta-vp.ru/news/./item/16715 downloaded successfully
http://g